In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize

In [2]:
df = pd.read_csv("clean_scout.csv")

In [3]:
df.shape

(15919, 33)

In [4]:
df.head(3).T

,0,1,2
make_model,Audi A1,Audi A1,Audi A1
body_type,Sedans,Sedans,Sedans
price,15770,14500,14640
vat,VAT deductible,Price negotiable,VAT deductible
km,56013.0,80000.0,83450.0
Type,Used,Used,Used
Warranty,NaN,NaN,NaN
Cylinders,3.0,4.0,NaN
Fuel,Diesel,Benzine,Diesel
Gears,NaN,7.0,NaN


In [5]:
df.isnull().sum()/df.shape[0]*100

make_model              0.000000
body_type               0.376908
price                   0.000000
vat                    28.349771
km                      6.432565
Type                    0.012564
Warranty               69.514417
Cylinders              35.680633
Fuel                    0.000000
Gears                  29.599849
Comfort_Convenience     5.779257
Entertainment_Media     8.631195
Extras                 18.606696
Safety_Security         6.168729
age                    10.032037
Previous_Owners        41.711163
hp_kW                   0.552799
Inspection_new         75.299956
Body_Color              3.750236
Paint_Type             36.258559
Upholstery_type        30.598656
Upholstery_color       31.898989
Nr_of_Doors             1.331742
Nr_of_Seats             6.137320
Gearing_Type            0.000000
Displacement_cc         3.115774
Weight_kg              43.809285
Drive_chain            43.080596
cons_comb              12.770903
cons_city              15.302469
cons_count

In [6]:
miss_val = []

[miss_val.append(i) for i in df.columns if any(df[i].isnull())]

miss_val

['body_type',
 'vat',
 'km',
 'Type',
 'Warranty',
 'Cylinders',
 'Gears',
 'Comfort_Convenience',
 'Entertainment_Media',
 'Extras',
 'Safety_Security',
 'age',
 'Previous_Owners',
 'hp_kW',
 'Inspection_new',
 'Body_Color',
 'Paint_Type',
 'Upholstery_type',
 'Upholstery_color',
 'Nr_of_Doors',
 'Nr_of_Seats',
 'Displacement_cc',
 'Weight_kg',
 'Drive_chain',
 'cons_comb',
 'cons_city',
 'cons_country',
 'CO2_Emission',
 'Emission_Class']

In [7]:
def first_looking(col):
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum()/df.shape[0]*100, 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].nunique())
    print("--------------------------------")
    print(df[col].value_counts(dropna = False))

In [8]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [9]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [10]:
def fill(df, group_col1, group_col2, col_name, method): # method can be "mode" or "median" or "ffill"
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
                
    elif method == "median":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].median()).fillna(df[cond1][col_name].median()).fillna(df[col_name].median())
                
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [11]:
first_looking("age")

column name    :  age
--------------------------------
per_of_nulls   :  % 10.03
num_of_nulls   :  1597
num_of_uniques :  4
--------------------------------
1.0    4522
3.0    3674
2.0    3273
0.0    2853
NaN    1597
Name: age, dtype: int64


In [12]:
df['age'].fillna('-', inplace=True)

In [13]:
df["age"].value_counts(dropna=False)

1.0    4522
3.0    3674
2.0    3273
0.0    2853
-      1597
Name: age, dtype: int64

In [14]:
df.groupby("age").km.describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
0.0,2706.0,2085.355137,5365.880693,1.0,10.00,50.0,3000.0,127022.0
1.0,4484.0,18035.239072,11052.524314,1.0,9990.00,17872.0,25078.5,136000.0
2.0,3272.0,41754.940709,28295.747886,1.0,21541.75,34752.0,54805.5,317000.0
3.0,3674.0,77442.520958,39170.142977,10.0,48000.00,72914.5,99950.0,291800.0
-,759.0,934.496706,7416.243875,0.0,5.00,10.0,10.0,89982.0


In [15]:
df[df["age"]=="-"]["km"].value_counts(dropna=False)

NaN        838
10.0       369
1.0        146
5.0         58
20.0        32
15.0        21
0.0         19
11.0        12
8.0         11
50.0        10
12.0         8
100.0        8
7.0          7
3.0          4
9.0          4
4.0          3
250.0        3
25.0         3
30.0         3
3000.0       2
39962.0      2
2.0          2
22627.0      2
784.0        1
89692.0      1
3500.0       1
325.0        1
497.0        1
99.0         1
77.0         1
40.0         1
19500.0      1
6100.0       1
11000.0      1
89982.0      1
4307.0       1
141.0        1
34164.0      1
500.0        1
150.0        1
11200.0      1
20768.0      1
32084.0      1
142.0        1
81800.0      1
281.0        1
6.0          1
68485.0      1
85000.0      1
196.0        1
4500.0       1
60.0         1
5000.0       1
Name: km, dtype: int64

In [16]:
cond1 = (df['km'] < 10000)
cond2 = ((df['km'] >= 10000) & (df['km'] < 28000))
cond3 = ((df['km'] >= 28000) & (df['km'] < 50000))
cond4 = (df['km'] >= 50000)

In [17]:
df.loc[cond1,'age'] = df.loc[cond1,'age'].replace('-', 0)
df.loc[cond2,'age'] = df.loc[cond2,'age'].replace('-', 1)
df.loc[cond3,'age'] = df.loc[cond3,'age'].replace('-', 2)
df.loc[cond4,'age'] = df.loc[cond4,'age'].replace('-', 3)

In [18]:
df.groupby('age').km.mean()

age
0.0     1647.362609
1.0    18035.130067
2.0    41748.576923
3.0    77450.062789
-               NaN
Name: km, dtype: float64

In [19]:
df["age"].value_counts(dropna=False)

1.0    4528
3.0    3679
0.0    3597
2.0    3277
-       838
Name: age, dtype: int64

In [20]:
df.groupby(['make_model',"body_type", 'age']).price.describe()

count          mean          std      min  \
make_model     body_type age                                              
Audi A1        Compact   0.0  198.0  23277.434343  3510.405726  14900.0   
                         1.0  268.0  18596.041045  2659.909876  13980.0   
                         2.0  161.0  16602.807453  2085.383788  10999.0   
                         3.0  234.0  14532.910256  1908.908968   9950.0   
                         -    178.0  23996.264045  3383.852477  16220.0   
...                             ...           ...          ...      ...   
Renault Espace Van       0.0  218.0  40074.513761  8154.729111  24200.0   
                         1.0  207.0  31947.840580  4072.970330  23099.0   
                         2.0  134.0  26574.477612  3427.862590  14600.0   
                         3.0  199.0  22132.623116  3216.031594  12614.0   
                         -     19.0  39632.315789  6866.138589  28650.0   

                                  25%      50%       75%      max  
make_model     body_type age                                       
Audi A1        Compact   0.0  20503.5  22492.0  26798.50  31990.0  
                         1.0  16445.0  16980.0  20950.00  23829.0  
                         2.0  15450.0  15850.0  17700.00  22150.0  
                         3.0  13407.5  13994.5  15480.00  18900.0  
                         -    21515.0  22875.0  27380.00  29181.0  
...                               ...      ...       ...      ...  
Renault Espace Van       0.0  32975.0  38925.0  45667.25  68320.0  
                         1.0  29385.0  31900.0  34990.00  38990.0  
                         2.0  24982.5  25490.0  28500.00  33800.0  
                         3.0  19900.0  22222.0  24650.00  27900.0  
                         -    33848.0  39550.0  45950.50  50170.0  

[176 rows x 8 columns]

In [21]:
df['age'].replace('-',0, inplace=True)

In [22]:
df.groupby('age').km.mean()

age
0.0     1647.362609
1.0    18035.130067
2.0    41748.576923
3.0    77450.062789
Name: km, dtype: float64

In [23]:
df["age"].value_counts(dropna=False)

1.0    4528
0.0    4435
3.0    3679
2.0    3277
Name: age, dtype: int64

In [24]:
first_looking("km")

column name    :  km
--------------------------------
per_of_nulls   :  % 6.43
num_of_nulls   :  1024
num_of_uniques :  6689
--------------------------------
10.0       1045
NaN        1024
1.0         367
5.0         170
50.0        148
           ... 
67469.0       1
43197.0       1
10027.0       1
35882.0       1
57.0          1
Name: km, Length: 6690, dtype: int64


In [25]:
df.groupby("age").km.mean()

age
0.0     1647.362609
1.0    18035.130067
2.0    41748.576923
3.0    77450.062789
Name: km, dtype: float64

In [26]:
df.groupby("age").km.transform("mean")

0        77450.062789
1        41748.576923
2        77450.062789
3        77450.062789
4        77450.062789
             ...     
15914     1647.362609
15915     1647.362609
15916     1647.362609
15917     1647.362609
15918     1647.362609
Name: km, Length: 15919, dtype: float64

In [27]:
df["km"].fillna(df.groupby("age").km.transform("mean"), inplace=True)

In [28]:
df.km.value_counts(dropna=False)

10.000000       1045
1647.362609      985
1.000000         367
5.000000         170
50.000000        148
                ... 
67469.000000       1
43197.000000       1
10027.000000       1
35882.000000       1
57.000000          1
Name: km, Length: 6692, dtype: int64

In [29]:
first_looking("body_type")

column name    :  body_type
--------------------------------
per_of_nulls   :  % 0.38
num_of_nulls   :  60
num_of_uniques :  9
--------------------------------
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


In [30]:
df.body_type.replace("Other", np.nan, inplace=True) 

In [31]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
NaN               350
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [32]:
df["body_type"].mode()

0    Sedans
Name: body_type, dtype: object

In [33]:
df["body_type"].mode()[0]

'Sedans'

In [34]:
for group in list(df["make_model"].unique()):
    cond = df["make_model"]==group
    mode = list(df[cond]["body_type"].mode())
    if mode != []:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df[cond]["body_type"].mode()[0])
    else:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df["body_type"].mode()[0])

In [35]:
df['body_type'].value_counts(dropna=False)

Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [36]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [37]:
fill_most(df, "make_model", "body_type")

Number of NaN :  0
------------------
Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


In [38]:
first_looking("Previous_Owners")

column name    :  Previous_Owners
--------------------------------
per_of_nulls   :  % 41.71
num_of_nulls   :  6640
num_of_uniques :  5
--------------------------------
1.0    8294
NaN    6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64


In [39]:
df["Previous_Owners"].fillna("-", inplace = True)

In [40]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
-      6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [41]:
df.groupby(['make_model', 'age', 'Previous_Owners']).km.describe()

count          mean           std  \
make_model     age Previous_Owners                                      
Audi A1        0.0 0.0               45.0    958.365063    815.361604   
                   1.0              239.0   3069.542783   3460.401760   
                   2.0                1.0   3000.000000           NaN   
                   -                521.0   1549.991910   2303.799717   
               1.0 0.0                1.0  15000.000000           NaN   
...                                   ...           ...           ...   
Renault Espace 3.0 1.0              118.0  79365.805085  36296.879893   
                   2.0               20.0  55316.250000  27356.183080   
                   3.0                1.0  40000.000000           NaN   
                   4.0                1.0  57000.000000           NaN   
                   -                159.0  73332.056604  32702.199643   

                                        min       25%           50%  \
make_model     age Previous_Owners                                    
Audi A1        0.0 0.0                  0.0     10.00   1647.362609   
                   1.0                  1.0     20.00   2500.000000   
                   2.0               3000.0   3000.00   3000.000000   
                   -                    1.0     11.00   1647.362609   
               1.0 0.0              15000.0  15000.00  15000.000000   
...                                     ...       ...           ...   
Renault Espace 3.0 1.0               3532.0  55000.00  76475.000000   
                   2.0               7300.0  42536.75  51972.500000   
                   3.0              40000.0  40000.00  40000.000000   
                   4.0              57000.0  57000.00  57000.000000   
                   -                 6119.0  50027.00  68570.000000   

                                             75%            max  
make_model     age Previous_Owners                               
Audi A1        0.0 0.0               1647.362609    1647.362609  
                   1.0               5000.000000   18000.000000  
                   2.0               3000.000000    3000.000000  
                   -                 1647.362609   15500.000000  
               1.0 0.0              15000.000000   15000.000000  
...                                          ...            ...  
Renault Espace 3.0 1.0              98375.750000  197414.000000  
                   2.0              64921.250000  148000.000000  
                   3.0              40000.000000   40000.000000  
                   4.0              57000.000000   57000.000000  
                   -                94000.000000  169068.000000  

[120 rows x 8 columns]

In [42]:
df[(df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")]["km"]

14894    1647.362609
14895    1647.362609
14896     101.000000
14897    1647.362609
14898     101.000000
14899     101.000000
14900    1647.362609
14901     101.000000
14903     101.000000
14904    1647.362609
14905    1647.362609
14906     101.000000
14907     101.000000
14908    1647.362609
14909    1647.362609
14910     101.000000
14911    1647.362609
14912     101.000000
14913    1647.362609
14914     101.000000
14915     101.000000
14916     101.000000
14917    1647.362609
14918    1647.362609
14919    1647.362609
14920     101.000000
14921    1647.362609
14922    1647.362609
14923     101.000000
14924    1647.362609
14925     101.000000
14926     101.000000
14927    1647.362609
Name: km, dtype: float64

In [43]:
cond = (df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")
df.loc[cond, "Previous_Owners"] = 0.0

In [44]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
-      6607
2.0     778
0.0     221
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [45]:
df["Previous_Owners"].replace("-", np.nan, inplace=True)

In [46]:
for group in list(df["age"].unique()):
    cond = df["age"]==group
    df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].fillna(method="ffill").fillna(method="bfill")
df["Previous_Owners"] = df["Previous_Owners"].fillna(method="ffill").fillna(method="bfill")

In [47]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    14162
2.0     1172
0.0      554
3.0       29
4.0        2
Name: Previous_Owners, dtype: int64

In [48]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [49]:
fill_prop(df, "age", "Previous_Owners")

Number of NaN :  0
------------------
1.0    14162
2.0     1172
0.0      554
3.0       29
4.0        2
Name: Previous_Owners, dtype: int64


In [50]:
first_looking("Warranty")

column name    :  Warranty
--------------------------------
per_of_nulls   :  % 69.51
num_of_nulls   :  11066
num_of_uniques :  41
--------------------------------
NaN     11066
12.0     2594
24.0     1118
60.0      401
36.0      279
48.0      149
6.0       125
72.0       59
3.0        33
23.0       11
18.0       10
20.0        7
25.0        6
2.0         5
50.0        4
26.0        4
16.0        4
4.0         3
1.0         3
19.0        3
34.0        3
13.0        3
28.0        2
22.0        2
14.0        2
11.0        2
46.0        2
21.0        2
9.0         2
17.0        2
45.0        2
33.0        1
40.0        1
65.0        1
10.0        1
15.0        1
7.0         1
8.0         1
56.0        1
49.0        1
47.0        1
30.0        1
Name: Warranty, dtype: int64


In [51]:
df["Warranty"].fillna("-", inplace = True)

In [52]:
df.groupby(['make_model', 'age', 'Warranty']).price.describe()

count          mean          std      min  \
make_model     age Warranty                                              
Audi A1        0.0 12.0       38.0  23846.342105  2978.849554  17900.0   
                   18.0        1.0  19995.000000          NaN  19995.0   
                   24.0      107.0  23766.308411  3338.105537  14900.0   
                   36.0       26.0  25514.423077  3205.835906  19900.0   
                   48.0       18.0  24441.666667  2516.781909  19650.0   
...                            ...           ...          ...      ...   
Renault Espace 3.0 24.0        4.0  24906.250000   827.177681  23890.0   
                   40.0        1.0  22500.000000          NaN  22500.0   
                   48.0        1.0  19490.000000          NaN  19490.0   
                   60.0        1.0  23890.000000          NaN  23890.0   
                   -         161.0  21931.559006  3562.319321  12614.0   

                                  25%      50%       75%      max  
make_model     age Warranty                                        
Audi A1        0.0 12.0      21000.00  23650.0  25725.00  28990.0  
                   18.0      19995.00  19995.0  19995.00  19995.0  
                   24.0      21890.00  22900.0  26890.00  32000.0  
                   36.0      22489.25  26400.0  28677.50  29179.0  
                   48.0      22800.00  25300.0  26200.00  28500.0  
...                               ...      ...       ...      ...  
Renault Espace 3.0 24.0      24535.00  24925.0  25296.25  25885.0  
                   40.0      22500.00  22500.0  22500.00  22500.0  
                   48.0      19490.00  19490.0  19490.00  19490.0  
                   60.0      23890.00  23890.0  23890.00  23890.0  
                   -         19013.00  21900.0  24899.00  30000.0  

[259 rows x 8 columns]

In [53]:
df.groupby(["make_model","Warranty"]).price.describe().sort_values(by = ["make_model", "mean"], ascending = False)

count          mean          std      min      25%  \
make_model     Warranty                                                       
Renault Espace 48.0       14.0  39806.785714  8058.500435  19490.0  37035.0   
               16.0        1.0  38890.000000          NaN  38890.0  38890.0   
               45.0        1.0  37950.000000          NaN  37950.0  37950.0   
               23.0        1.0  37600.000000          NaN  37600.0  37600.0   
               25.0        1.0  36498.000000          NaN  36498.0  36498.0   
...                        ...           ...          ...      ...      ...   
Audi A1        23.0        2.0  16560.000000  6208.397539  12170.0  14365.0   
               47.0        1.0  16370.000000          NaN  16370.0  16370.0   
               49.0        1.0  16270.000000          NaN  16270.0  16270.0   
               46.0        1.0  15980.000000          NaN  15980.0  15980.0   
               26.0        1.0  15840.000000          NaN  15840.0  15840.0   

                             50%      75%      max  
make_model     Warranty                             
Renault Espace 48.0      40692.5  46095.0  47990.0  
               16.0      38890.0  38890.0  38890.0  
               45.0      37950.0  37950.0  37950.0  
               23.0      37600.0  37600.0  37600.0  
               25.0      36498.0  36498.0  36498.0  
...                          ...      ...      ...  
Audi A1        23.0      16560.0  18755.0  20950.0  
               47.0      16370.0  16370.0  16370.0  
               49.0      16270.0  16270.0  16270.0  
               46.0      15980.0  15980.0  15980.0  
               26.0      15840.0  15840.0  15840.0  

[130 rows x 8 columns]

In [54]:
df.drop("Warranty", axis=1, inplace=True)

In [55]:
first_looking("vat")

column name    :  vat
--------------------------------
per_of_nulls   :  % 28.35
num_of_nulls   :  4513
num_of_uniques :  2
--------------------------------
VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64


In [56]:
df.vat.fillna(method="ffill", inplace = True)

In [57]:
df.vat.value_counts(dropna=False)

VAT deductible      15048
Price negotiable      871
Name: vat, dtype: int64

In [58]:
first_looking("Body_Color")

column name    :  Body_Color
--------------------------------
per_of_nulls   :  % 3.75
num_of_nulls   :  597
num_of_uniques :  14
--------------------------------
Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: Body_Color, dtype: int64


In [59]:
df["Body_Color"].fillna("-", inplace = True)

In [60]:
df["Body_Color"].value_counts(dropna=False)

Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
-          597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: Body_Color, dtype: int64

In [61]:
df.groupby(["make_model", "body_type", 'Body_Color']).price.describe()

count          mean          std  \
make_model     body_type Body_Color                                     
Audi A1        Compact   -            44.0  20291.840909  3456.615867   
                         Beige         6.0  20556.500000  2475.511563   
                         Black       320.0  18196.281250  4206.972661   
                         Blue         96.0  19145.406250  4541.862986   
                         Brown         9.0  16982.000000  2964.387753   
...                                    ...           ...          ...   
Renault Espace Van       Brown        27.0  25718.259259  9649.962266   
                         Grey        301.0  30236.724252  8635.879175   
                         Silver       36.0  27373.000000  4809.681926   
                         Violet       11.0  24434.272727  4603.886230   
                         White       175.0  32843.680000  7568.938010   

                                         min       25%      50%      75%  \
make_model     body_type Body_Color                                        
Audi A1        Compact   -           15100.0  17844.00  19944.5  22020.0   
                         Beige       16240.0  19766.75  21420.0  21700.0   
                         Black        9950.0  14990.00  16890.0  21390.0   
                         Blue        11444.0  15870.00  16925.0  22226.0   
                         Brown       11445.0  15993.00  16820.0  18850.0   
...                                      ...       ...      ...      ...   
Renault Espace Van       Brown       12614.0  19225.00  22990.0  27187.5   
                         Grey        15500.0  24800.00  28000.0  34500.0   
                         Silver      12990.0  24560.00  28500.0  29900.0   
                         Violet      19900.0  20499.50  23900.0  26299.0   
                         White       12990.0  29237.50  31975.0  35886.5   

                                         max  
make_model     body_type Body_Color           
Audi A1        Compact   -           29181.0  
                         Beige       23250.0  
                         Black       28997.0  
                         Blue        28980.0  
                         Brown       20750.0  
...                                      ...  
Renault Espace Van       Brown       47990.0  
                         Grey        64332.0  
                         Silver      35100.0  
                         Violet      34990.0  
                         White       64298.0  

[244 rows x 8 columns]

In [62]:
df.drop("Body_Color", axis=1, inplace=True)

In [63]:
first_looking("Paint_Type")

column name    :  Paint_Type
--------------------------------
per_of_nulls   :  % 36.26
num_of_nulls   :  5772
num_of_uniques :  3
--------------------------------
Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: Paint_Type, dtype: int64


In [64]:
df["Paint_Type"].fillna("-", inplace = True)

In [65]:
df["Paint_Type"].value_counts(dropna=False)

Metallic       9794
-              5772
Uni/basic       347
Perl effect       6
Name: Paint_Type, dtype: int64

In [66]:
df.groupby(["make_model", "body_type", "age", 'Paint_Type']).price.describe()

count          mean          std  \
make_model     body_type age Paint_Type                                     
Audi A1        Compact   0.0 -           145.0  22082.717241  3341.754389   
                             Metallic    229.0  24622.257642  3172.063399   
                             Uni/basic     2.0  19888.000000     0.000000   
                         1.0 -            56.0  17590.517857  2369.313410   
                             Metallic    212.0  18861.650943  2674.019324   
...                                        ...           ...          ...   
Renault Espace Van       2.0 Metallic     91.0  26799.296703  3337.258344   
                             Uni/basic     1.0  29200.000000          NaN   
                         3.0 -            67.0  21713.149254  3450.674178   
                             Metallic    136.0  22317.029412  3286.732267   
                             Uni/basic     3.0  22628.333333  2094.721541   

                                             min       25%      50%       75%  \
make_model     body_type age Paint_Type                                         
Audi A1        Compact   0.0 -           14900.0  19850.00  21490.0  23475.00   
                             Metallic    17880.0  21990.00  23700.0  27780.00   
                             Uni/basic   19888.0  19888.00  19888.0  19888.00   
                         1.0 -           14220.0  15852.25  16864.5  18367.25   
                             Metallic    13980.0  16448.75  17515.0  21482.50   
...                                          ...       ...      ...       ...   
Renault Espace Van       2.0 Metallic    16950.0  25490.00  25490.0  28495.00   
                             Uni/basic   29200.0  29200.00  29200.0  29200.00   
                         3.0 -           12990.0  19195.00  22900.0  23490.00   
                             Metallic    12614.0  19900.00  21990.0  24912.50   
                             Uni/basic   20995.0  21447.50  21900.0  23445.00   

                                             max  
make_model     body_type age Paint_Type           
Audi A1        Compact   0.0 -           29181.0  
                             Metallic    31990.0  
                             Uni/basic   19888.0  
                         1.0 -           23650.0  
                             Metallic    23829.0  
...                                          ...  
Renault Espace Van       2.0 Metallic    33800.0  
                             Uni/basic   29200.0  
                         3.0 -           32900.0  
                             Metallic    27900.0  
                             Uni/basic   24990.0  

[272 rows x 8 columns]

In [67]:
df.groupby(["make_model", "body_type", 'Paint_Type']).price.describe().sort_values(by = ["make_model", "body_type", "mean"], ascending = False)

count          mean          std  \
make_model     body_type   Paint_Type                                     
Renault Espace Van         Metallic    505.0  31507.811881  9533.340846   
                           -           293.0  30015.815700  7147.545230   
                           Uni/basic    13.0  27971.153846  4495.663723   
               Transporter Metallic     34.0  22865.823529  4255.807045   
                           -            18.0  22116.611111  4248.690001   
...                                      ...           ...          ...   
Audi A1        Sedans      -           493.0  18104.022312  4529.704639   
               Coupe       Metallic      2.0  14925.000000  1378.858223   
               Compact     Metallic    723.0  19412.383126  4665.034898   
                           -           311.0  18709.099678  4273.170450   
                           Uni/basic     5.0  16815.200000  3405.155386   

                                           min       25%      50%      75%  \
make_model     body_type   Paint_Type                                        
Renault Espace Van         Metallic    12614.0  24950.00  29500.0  36781.0   
                           -           12990.0  24790.00  30965.0  34070.0   
                           Uni/basic   20995.0  24750.00  26900.0  31900.0   
               Transporter Metallic    14999.0  19924.75  21750.0  24975.0   
                           -           16900.0  19575.00  20750.0  25250.0   
...                                        ...       ...      ...      ...   
Audi A1        Sedans      -            8999.0  14300.00  17300.0  20990.0   
               Coupe       Metallic    13950.0  14437.50  14925.0  15412.5   
               Compact     Metallic     9950.0  15740.00  18750.0  22490.0   
                           -           10490.0  15850.00  17950.0  21490.0   
                           Uni/basic   12900.0  13500.00  17900.0  19888.0   

                                           max  
make_model     body_type   Paint_Type           
Renault Espace Van         Metallic    68320.0  
                           -           54205.0  
                           Uni/basic   34000.0  
               Transporter Metallic    31000.0  
                           -           32700.0  
...                                        ...  
Audi A1        Sedans      -           37900.0  
               Coupe       Metallic    15900.0  
               Compact     Metallic    31990.0  
                           -           29181.0  
                           Uni/basic   19888.0  

[98 rows x 8 columns]

In [68]:
df["Paint_Type"].replace("-", np.nan, inplace = True)

In [69]:
df["Paint_Type"].value_counts(dropna=False)

Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: Paint_Type, dtype: int64

In [70]:
for group1 in list(df["make_model"].unique()):
    for group2 in list(df["body_type"].unique()):
        cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
        df.loc[cond2, "Paint_Type"] = df.loc[cond2, "Paint_Type"].fillna(method="ffill").fillna(method="bfill")
                
for group1 in list(df["make_model"].unique()):
    cond1 = df["make_model"]==group1
    df.loc[cond1, "Paint_Type"] = df.loc[cond1, "Paint_Type"].fillna(method="ffill").fillna(method="bfill")            
           
df["Paint_Type"] = df["Paint_Type"].fillna(method="ffill").fillna(method="bfill")

In [71]:
df["Paint_Type"].value_counts(dropna=False)

Metallic       15250
Uni/basic        637
Perl effect       32
Name: Paint_Type, dtype: int64

In [72]:
def fill_ffill(df, group_col1, group_col2, col_name):
    
    for group1 in list(df[group_col1].unique()):
        for group2 in list(df[group_col2].unique()):
            cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
            df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
    for group1 in list(df[group_col1].unique()):
        cond1 = df[group_col1]==group1
        df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")

In [73]:
fill_ffill(df, "make_model", "body_type", "Paint_Type")

In [74]:
df["Paint_Type"].value_counts(dropna=False)

Metallic       15250
Uni/basic        637
Perl effect       32
Name: Paint_Type, dtype: int64

In [75]:
first_looking("Type")

column name    :  Type
--------------------------------
per_of_nulls   :  % 0.01
num_of_nulls   :  2
num_of_uniques :  5
--------------------------------
Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: Type, dtype: int64


In [76]:
df.Type.fillna("-", inplace=True)

In [77]:
df['Type'].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
-                     2
Name: Type, dtype: int64

In [78]:
df.groupby(["Type", "make_model", "age"]).km.describe()

count           mean           std  \
Type          make_model     age                                       
-             Audi A3        0.0    1.0    1647.362609           NaN   
                             3.0    1.0  115137.000000           NaN   
Demonstration Audi A1        0.0  110.0    3784.227273   2498.414744   
                             1.0   19.0    9931.157895   5209.543265   
                             2.0    6.0   22963.500000  17802.676830   
...                                 ...            ...           ...   
Used          Renault Duster 0.0   16.0     101.000000      0.000000   
              Renault Espace 0.0   40.0    2101.025000   3625.577998   
                             1.0  160.0   16368.250813  11001.737792   
                             2.0  154.0   44076.110390  27116.402205   
                             3.0  299.0   74481.993311  34217.258314   

                                            min            25%            50%  \
Type          make_model     age                                                
-             Audi A3        0.0    1647.362609    1647.362609    1647.362609   
                             3.0  115137.000000  115137.000000  115137.000000   
Demonstration Audi A1        0.0      10.000000    2541.250000    3000.500000   
                             1.0    1050.000000    5500.000000    9522.000000   
                             2.0   11940.000000   15327.250000   16378.500000   
...                                         ...            ...            ...   
Used          Renault Duster 0.0     101.000000     101.000000     101.000000   
              Renault Espace 0.0      10.000000      10.000000     100.000000   
                             1.0      10.000000    9345.500000   15000.000000   
                             2.0    5600.000000   25326.500000   44677.500000   
                             3.0    3532.000000   49950.000000   68900.000000   

                                            75%            max  
Type          make_model     age                                
-             Audi A3        0.0    1647.362609    1647.362609  
                             3.0  115137.000000  115137.000000  
Demonstration Audi A1        0.0    4999.750000   12700.000000  
                             1.0   14560.000000   18900.000000  
                             2.0   18425.000000   59000.000000  
...                                         ...            ...  
Used          Renault Duster 0.0     101.000000     101.000000  
              Renault Espace 0.0    2071.000000   15400.000000  
                             1.0   23002.500000   62860.000000  
                             2.0   49606.000000  240000.000000  
                             3.0   95000.000000  197414.000000  

[113 rows x 8 columns]

In [79]:
cond1 = (df['make_model'] == "Audi A3") & (df["age"] == 0)
cond2 = (df['make_model'] == "Audi A3") & (df["age"] == 3)

In [80]:
df.loc[cond1,'Type'] = df.loc[cond1,'Type'].replace('-','New')
df.loc[cond2,'Type'] = df.loc[cond2,'Type'].replace('-','Used')

In [81]:
df['Type'].value_counts(dropna=False)

Used              11097
New                1651
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: Type, dtype: int64

In [82]:
first_looking("Inspection_new")

column name    :  Inspection_new
--------------------------------
per_of_nulls   :  % 75.3
num_of_nulls   :  11987
num_of_uniques :  1
--------------------------------
NaN    11987
Yes     3932
Name: Inspection_new, dtype: int64


In [83]:
df["Inspection_new"].fillna("-", inplace=True)

In [84]:
df["Inspection_new"].value_counts(dropna=False)

-      11987
Yes     3932
Name: Inspection_new, dtype: int64

In [85]:
df.groupby(["make_model", "body_type", "age", "Inspection_new"]).price.describe()

count          mean          std  \
make_model     body_type age Inspection_new                                     
Audi A1        Compact   0.0 -               243.0  24206.370370  3530.094659   
                             Yes             133.0  22542.248120  3076.119339   
                         1.0 -               150.0  19065.306667  2836.071257   
                             Yes             118.0  17999.516949  2293.775840   
                         2.0 -               115.0  16425.669565  1985.790164   
...                                            ...           ...          ...   
Renault Espace Van       1.0 Yes              33.0  33000.878788  3414.040639   
                         2.0 -               137.0  26507.087591  3422.048171   
                             Yes               2.0  27139.500000   211.424928   
                         3.0 -               176.0  22182.267045  3270.032173   
                             Yes              30.0  21790.100000  3690.170572   

                                                 min       25%      50%  \
make_model     body_type age Inspection_new                               
Audi A1        Compact   0.0 -               16220.0  21435.00  23480.0   
                             Yes             14900.0  20388.00  22400.0   
                         1.0 -               14500.0  16490.00  18800.0   
                             Yes             13980.0  16445.00  16910.0   
                         2.0 -               10999.0  15450.00  15850.0   
...                                              ...       ...      ...   
Renault Espace Van       1.0 Yes             29385.0  29990.00  31899.0   
                         2.0 -               14600.0  24980.00  25490.0   
                             Yes             26990.0  27064.75  27139.5   
                         3.0 -               12990.0  19900.00  22261.0   
                             Yes             12614.0  19085.00  21982.5   

                                                  75%      max  
make_model     body_type age Inspection_new                     
Audi A1        Compact   0.0 -               27390.00  31990.0  
                             Yes             22950.00  29197.0  
                         1.0 -               21949.75  23777.0  
                             Yes             19963.50  23829.0  
                         2.0 -               16890.00  22150.0  
...                                               ...      ...  
Renault Espace Van       1.0 Yes             35300.00  38990.0  
                         2.0 -               28500.00  33800.0  
                             Yes             27214.25  27289.0  
                         3.0 -               24550.00  32900.0  
                             Yes             24825.00  26970.0  

[199 rows x 8 columns]

In [86]:
df["Inspection_new"].replace("-", "No", inplace=True)

In [87]:
df["Inspection_new"].value_counts(dropna=False)

No     11987
Yes     3932
Name: Inspection_new, dtype: int64

In [89]:
df["Inspection_new"].replace(["Yes", "No"], [1,0], inplace = True)

In [90]:
df["Inspection_new"].value_counts(dropna=False)

0    11987
1     3932
Name: Inspection_new, dtype: int64

In [109]:
first_looking("Upholstery_type")

column name    :  Upholstery_type
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  5
--------------------------------
Cloth           12157
Part leather     2128
Full leather     1458
alcantara          95
Velour             81
Name: Upholstery_type, dtype: int64


In [101]:
df["Upholstery_type"].replace(["Velour", "alcantara", "Part leather", "Full leather"], ["Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"])

0                    Cloth
1                    Cloth
2                    Cloth
3                    Cloth
4                    Cloth
               ...        
15914    Part/Full Leather
15915                Cloth
15916    Part/Full Leather
15917    Part/Full Leather
15918    Part/Full Leather
Name: Upholstery_type, Length: 15919, dtype: object

In [107]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth           12157
Part leather     2128
Full leather     1458
alcantara          95
Velour             81
Name: Upholstery_type, dtype: int64

In [108]:
fill(df, "make_model", "body_type", "Upholstery_type", "ffill")

Number of NaN :  0
------------------
Cloth           12157
Part leather     2128
Full leather     1458
alcantara          95
Velour             81
Name: Upholstery_type, dtype: int64


In [110]:
df.drop("Upholstery_color", axis=1, inplace=True)

In [111]:
first_looking("Nr_of_Doors")

column name    :  Nr_of_Doors
--------------------------------
per_of_nulls   :  % 1.33
num_of_nulls   :  212
num_of_uniques :  6
--------------------------------
5.0    11575
4.0     3079
3.0      832
2.0      219
NaN      212
1.0        1
7.0        1
Name: Nr_of_Doors, dtype: int64


In [112]:
fill(df, "make_model", "body_type", "Nr_of_Doors", "mode")

Number of NaN :  0
------------------
5.0    11787
4.0     3079
3.0      832
2.0      219
1.0        1
7.0        1
Name: Nr_of_Doors, dtype: int64


In [113]:
first_looking("Nr_of_Seats")

column name    :  Nr_of_Seats
--------------------------------
per_of_nulls   :  % 6.14
num_of_nulls   :  977
num_of_uniques :  6
--------------------------------
5.0    13336
4.0     1125
NaN      977
7.0      362
2.0      116
6.0        2
3.0        1
Name: Nr_of_Seats, dtype: int64


In [114]:
fill(df, "make_model", "body_type", "Nr_of_Seats", "mode")

Number of NaN :  0
------------------
5.0    14308
4.0     1127
7.0      362
2.0      119
6.0        2
3.0        1
Name: Nr_of_Seats, dtype: int64


In [115]:
first_looking("Cylinders")

column name    :  Cylinders
--------------------------------
per_of_nulls   :  % 35.68
num_of_nulls   :  5680
num_of_uniques :  7
--------------------------------
4.0    8105
NaN    5680
3.0    2104
5.0      22
6.0       3
8.0       2
2.0       2
1.0       1
Name: Cylinders, dtype: int64


In [116]:
fill(df, "make_model", "body_type", "Cylinders", "mode")

Number of NaN :  0
------------------
4.0    12926
3.0     2963
5.0       22
6.0        3
8.0        2
2.0        2
1.0        1
Name: Cylinders, dtype: int64


In [117]:
df.drop("Cylinders", axis = 1, inplace = True)

In [118]:
first_looking("Drive_chain")

column name    :  Drive_chain
--------------------------------
per_of_nulls   :  % 43.08
num_of_nulls   :  6858
num_of_uniques :  3
--------------------------------
front    8886
NaN      6858
4WD       171
rear        4
Name: Drive_chain, dtype: int64


In [121]:
df["Drive_chain"].fillna("-", inplace=True)

In [122]:
df.groupby(["make_model", "body_type", "Drive_chain"]).price.describe()

count          mean           std  \
make_model     body_type   Drive_chain                                      
Audi A1        Compact     -            352.0  17620.869318   4226.116239   
                           4WD            2.0  14790.000000   1258.650071   
                           front        685.0  20008.223358   4511.347852   
               Coupe       -              2.0  14925.000000   1378.858223   
               Sedans      -            561.0  17830.440285   4362.321407   
...                                       ...           ...           ...   
Renault Espace Transporter 4WD            1.0  30000.000000           NaN   
                           front          6.0  23258.166667   5973.317350   
               Van         -            394.0  29624.208122   8543.179966   
                           4WD           38.0  38150.763158  10013.779035   
                           front        379.0  31525.163588   8343.191759   

                                            min      25%      50%       75%  \
make_model     body_type   Drive_chain                                        
Audi A1        Compact     -            10490.0  14990.0  15900.0  20885.75   
                           4WD          13900.0  14345.0  14790.0  15235.00   
                           front         9950.0  16430.0  19890.0  22690.00   
               Coupe       -            13950.0  14437.5  14925.0  15412.50   
               Sedans      -             8999.0  14900.0  16490.0  20700.00   
...                                         ...      ...      ...       ...   
Renault Espace Transporter 4WD          30000.0  30000.0  30000.0  30000.00   
                           front        17000.0  19262.5  21500.0  26624.25   
               Van         -            12990.0  23990.0  27545.0  34368.75   
                           4WD          24950.0  31900.0  34200.0  45747.00   
                           front        12614.0  25050.0  31360.0  36151.00   

                                            max  
make_model     body_type   Drive_chain           
Audi A1        Compact     -            29190.0  
                           4WD          15680.0  
                           front        31990.0  
               Coupe       -            15900.0  
               Sedans      -            37900.0  
...                                         ...  
Renault Espace Transporter 4WD          30000.0  
                           front        32700.0  
               Van         -            63477.0  
                           4WD          57990.0  
                           front        68320.0  

[93 rows x 8 columns]

In [123]:
cond = (df['make_model'] == "Renault Duster") & (df["body_type"] == "Off-Road")

In [124]:
df.loc[cond,'Drive_chain'] = df.loc[cond,'Drive_chain'].replace('-','4WD')

In [125]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
-        6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [126]:
df["Drive_chain"] = df["Drive_chain"].replace('-', np.nan)

In [127]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
NaN      6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [128]:
fill(df, "make_model", "body_type", "Drive_chain", "mode")

Number of NaN :  0
------------------
front    15711
4WD        204
rear         4
Name: Drive_chain, dtype: int64


In [129]:
first_looking("Emission_Class")

column name    :  Emission_Class
--------------------------------
per_of_nulls   :  % 22.79
num_of_nulls   :  3628
num_of_uniques :  3
--------------------------------
Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64


In [130]:
df["Emission_Class"].fillna("-", inplace=True)

In [131]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
-          3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [132]:
df.groupby(["make_model", "age", "Fuel", "Emission_Class"]).price.describe()

count          mean          std  \
make_model     age Fuel    Emission_Class                                     
Audi A1        0.0 Benzine -               267.0  23770.318352  3136.484598   
                           Euro 6          535.0  23948.532710  3472.575062   
                   Diesel  Euro 6            4.0  19370.000000  1181.891140   
               1.0 Benzine -                99.0  19317.060606  2855.338208   
                           Euro 5            3.0  17862.666667  1840.592658   
...                                          ...           ...          ...   
Renault Espace 3.0 Benzine -                 3.0  22790.000000  3852.804174   
                           Euro 6           29.0  23773.413793  2649.283953   
                   Diesel  -                65.0  22353.030769  3158.169238   
                           Euro 5            1.0  24900.000000          NaN   
                           Euro 6          202.0  21425.391089  3281.749248   

                                               min       25%      50%  \
make_model     age Fuel    Emission_Class                               
Audi A1        0.0 Benzine -               14900.0  21875.00  22900.0   
                           Euro 6          15550.0  21285.00  22900.0   
                   Diesel  Euro 6          17900.0  18717.50  19495.0   
               1.0 Benzine -               14490.0  16684.50  19887.0   
                           Euro 5          16800.0  16800.00  16800.0   
...                                            ...       ...      ...   
Renault Espace 3.0 Benzine -               18480.0  21235.00  23990.0   
                           Euro 6          17900.0  21950.00  23890.0   
                   Diesel  -               14900.0  20745.00  22690.0   
                           Euro 5          24900.0  24900.00  24900.0   
                           Euro 6          12614.0  19034.75  20945.0   

                                               75%      max  
make_model     age Fuel    Emission_Class                    
Audi A1        0.0 Benzine -               26330.0  31990.0  
                           Euro 6          27107.5  37900.0  
                   Diesel  Euro 6          20147.5  20590.0  
               1.0 Benzine -               21900.0  23880.0  
                           Euro 5          18394.0  19988.0  
...                                            ...      ...  
Renault Espace 3.0 Benzine -               24945.0  25900.0  
                           Euro 6          25990.0  27900.0  
                   Diesel  -               23990.0  32900.0  
                           Euro 5          24900.0  24900.0  
                           Euro 6          23950.0  30000.0  

[162 rows x 8 columns]

In [133]:
df["Emission_Class"].replace("-", np.nan, inplace=True)

In [134]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [135]:
fill(df, "age", "Fuel", "Emission_Class", "ffill")

Number of NaN :  0
------------------
Euro 6    15769
Euro 5      100
Euro 4       50
Name: Emission_Class, dtype: int64


In [136]:
df.drop("Emission_Class", axis=1, inplace=True)

In [137]:
first_looking("Gears")

column name    :  Gears
--------------------------------
per_of_nulls   :  % 29.6
num_of_nulls   :  4712
num_of_uniques :  10
--------------------------------
6.0     5822
NaN     4712
5.0     3239
7.0     1908
8.0      224
9.0        6
1.0        2
3.0        2
4.0        2
2.0        1
50.0       1
Name: Gears, dtype: int64


In [139]:
df["Gears"].fillna("-", inplace=True)

In [140]:
df["Gears"].value_counts(dropna=False)

6.0     5822
-       4712
5.0     3239
7.0     1908
8.0      224
9.0        6
1.0        2
3.0        2
4.0        2
2.0        1
50.0       1
Name: Gears, dtype: int64

In [141]:
df.groupby(["make_model", "body_type", "Gearing_Type", "Gears"]).price.describe()

count          mean  \
make_model     body_type Gearing_Type   Gears                        
Audi A1        Compact   Automatic      5.0      3.0  22184.333333   
                                        6.0      6.0  21038.333333   
                                        7.0    199.0  22059.251256   
                                        8.0      1.0  16880.000000   
                                        -      253.0  21640.426877   
...                                              ...           ...   
Renault Espace Van       Manual         7.0      1.0  36590.000000   
                                        -       14.0  27123.214286   
                         Semi-automatic 6.0     11.0  28885.454545   
                                        7.0      1.0  19900.000000   
                                        -        4.0  26445.000000   

                                                        std      min      25%  \
make_model     body_type Gearing_Type   Gears                                   
Audi A1        Compact   Automatic      5.0     3421.301555  18497.0  20648.5   
                                        6.0     4282.038845  16430.0  18725.0   
                                        7.0     3918.115996  13990.0  18970.0   
                                        8.0             NaN  16880.0  16880.0   
                                        -       4965.499725  13880.0  16975.0   
...                                                     ...      ...      ...   
Renault Espace Van       Manual         7.0             NaN  36590.0  36590.0   
                                        -       8657.066222  16480.0  19623.0   
                         Semi-automatic 6.0    11439.109549  16900.0  21450.0   
                                        7.0             NaN  19900.0  19900.0   
                                        -       4549.289322  22900.0  23650.0   

                                                   50%      75%      max  
make_model     body_type Gearing_Type   Gears                             
Audi A1        Compact   Automatic      5.0    22800.0  24028.0  25256.0  
                                        6.0    20920.0  21060.0  28860.0  
                                        7.0    21790.0  24365.0  29181.0  
                                        8.0    16880.0  16880.0  16880.0  
                                        -      20950.0  26980.0  29197.0  
...                                                ...      ...      ...  
Renault Espace Van       Manual         7.0    36590.0  36590.0  36590.0  
                                        -      25740.0  32625.0  43080.0  
                         Semi-automatic 6.0    24900.0  32750.0  56950.0  
                                        7.0    19900.0  19900.0  19900.0  
                                        -      24945.0  27740.0  32990.0  

[215 rows x 8 columns]

In [142]:
df["Gears"].replace([1,2,3,4,9,50,"-"], np.nan, inplace=True)

In [143]:
df["Gears"].value_counts(dropna=False)

6.0    5822
NaN    4726
5.0    3239
7.0    1908
8.0     224
Name: Gears, dtype: int64

In [144]:
for group1 in list(df["make_model"].unique()):
    for group2 in list(df["body_type"].unique()):
        for group3 in list(df["Gearing_Type"].unique()):
            cond1 = df["make_model"]==group1
            cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
            cond3 = (df["make_model"]==group1) & (df["body_type"]==group2) & (df["Gearing_Type"]==group3)
            mode1 = list(df[cond1]["Gears"].mode())
            mode2 = list(df[cond2]["Gears"].mode())
            mode3 = list(df[cond3]["Gears"].mode())
            if mode3 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond3]["Gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond2]["Gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond1]["Gears"].mode()[0])
            else:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df["Gears"].mode()[0])

In [145]:
df["Gears"].value_counts(dropna=False)

6.0    8626
5.0    4258
7.0    2810
8.0     225
Name: Gears, dtype: int64

In [146]:
first_looking("hp_kW")

column name    :  hp_kW
--------------------------------
per_of_nulls   :  % 0.55
num_of_nulls   :  88
num_of_uniques :  80
--------------------------------
85.0     2542
66.0     2122
81.0     1402
100.0    1308
110.0    1112
         ... 
84.0        1
195.0       1
44.0        1
239.0       1
9.0         1
Name: hp_kW, Length: 81, dtype: int64


In [147]:
df["hp_kW"].fillna("-", inplace=True)

In [148]:
df.groupby(["make_model", "body_type","hp_kW"]).price.describe()

count          mean          std      min  \
make_model     body_type hp_kW                                              
Audi A1        Compact   60.0    58.0  15189.827586  1663.611419  10900.0   
                         66.0   162.0  15398.209877  1934.300783  10490.0   
                         70.0   332.0  17983.262048  3315.814057   9950.0   
                         71.0    32.0  20831.593750  2410.839780  15890.0   
                         85.0   330.0  22604.103030  4572.568357  11100.0   
...                               ...           ...          ...      ...   
Renault Espace Van       150.0    1.0  43545.000000          NaN  43545.0   
                         165.0  175.0  38205.960000  7911.484750  26800.0   
                         167.0    2.0  37990.000000  2828.427125  35990.0   
                         168.0    3.0  38410.000000  1712.658752  36590.0   
                         -       34.0  28466.029412  4898.732331  17990.0   

                                    25%      50%       75%      max  
make_model     body_type hp_kW                                       
Audi A1        Compact   60.0   14390.0  15774.5  16345.00  16978.0  
                         66.0   14042.5  15465.0  16445.00  23700.0  
                         70.0   15480.0  17447.0  21190.00  28990.0  
                         71.0   18937.5  21425.0  22462.25  25256.0  
                         85.0   19700.0  22497.0  26980.00  31990.0  
...                                 ...      ...       ...      ...  
Renault Espace Van       150.0  43545.0  43545.0  43545.00  43545.0  
                         165.0  31990.0  35989.0  40999.00  64332.0  
                         167.0  36990.0  37990.0  38990.00  39990.0  
                         168.0  37620.0  38650.0  39320.00  39990.0  
                         -      24515.0  28195.0  32900.00  39990.0  

[311 rows x 8 columns]

In [149]:
df["hp_kW"].replace("-", np.nan, inplace=True)

In [150]:
fill(df, "make_model", "body_type", "hp_kW", "mode")

Number of NaN :  0
------------------
85.0     2543
66.0     2124
81.0     1403
100.0    1314
110.0    1113
         ... 
123.0       1
44.0        1
195.0       1
75.0        1
9.0         1
Name: hp_kW, Length: 80, dtype: int64


In [151]:
first_looking("Displacement_cc")

column name    :  Displacement_cc
--------------------------------
per_of_nulls   :  % 3.12
num_of_nulls   :  496
num_of_uniques :  77
--------------------------------
1598.0    4761
999.0     2438
1398.0    1314
1399.0     749
1229.0     677
          ... 
122.0        1
1198.0       1
1195.0       1
2967.0       1
1800.0       1
Name: Displacement_cc, Length: 78, dtype: int64


In [152]:
df["Displacement_cc"].fillna("-", inplace=True)

In [153]:
df.groupby(["make_model", "body_type","Displacement_cc"]).price.describe()

count          mean          std  \
make_model     body_type Displacement_cc                                     
Audi A1        Compact   929.0              5.0  16364.000000  1504.818926   
                         995.0              2.0  21990.000000  1414.213562   
                         999.0            663.0  20294.101056  4570.798394   
                         1000.0             7.0  19498.428571  5396.195940   
                         1395.0            98.0  18246.622449  3084.080576   
...                                         ...           ...          ...   
Renault Espace Van       1798.0           174.0  38173.298851  7873.213630   
                         1800.0             1.0  50750.000000          NaN   
                         1997.0            92.0  39621.423913  7514.270423   
                         2000.0             1.0  21900.000000          NaN   
                         -                 77.0  28619.610390  5732.766495   

                                              min      25%      50%       75%  \
make_model     body_type Displacement_cc                                        
Audi A1        Compact   929.0            14980.0  15480.0  16240.0  16240.00   
                         995.0            20990.0  21490.0  21990.0  22490.00   
                         999.0             9950.0  16480.0  20380.0  22820.00   
                         1000.0           13500.0  16024.5  16800.0  22695.00   
                         1395.0           12550.0  15850.0  17835.0  20900.00   
...                                           ...      ...      ...       ...   
Renault Espace Van       1798.0           26800.0  31990.0  35990.0  40991.75   
                         1800.0           50750.0  50750.0  50750.0  50750.00   
                         1997.0           12990.0  32975.0  39625.0  46262.50   
                         2000.0           21900.0  21900.0  21900.0  21900.00   
                         -                17500.0  24490.0  28465.0  32935.00   

                                              max  
make_model     body_type Displacement_cc           
Audi A1        Compact   929.0            18880.0  
                         995.0            22990.0  
                         999.0            31990.0  
                         1000.0           28750.0  
                         1395.0           28880.0  
...                                           ...  
Renault Espace Van       1798.0           64332.0  
                         1800.0           50750.0  
                         1997.0           56950.0  
                         2000.0           21900.0  
                         -                44500.0  

[289 rows x 8 columns]

In [154]:
df["Displacement_cc"].replace("-", np.nan, inplace=True)

In [155]:
fill(df, "make_model", "body_type", "Displacement_cc", "mode")

Number of NaN :  0
------------------
1598.0    5044
999.0     2467
1398.0    1387
1399.0     768
1229.0     678
          ... 
122.0        1
1198.0       1
1195.0       1
2967.0       1
1800.0       1
Name: Displacement_cc, Length: 77, dtype: int64


In [156]:
first_looking("Weight_kg")

column name    :  Weight_kg
--------------------------------
per_of_nulls   :  % 43.81
num_of_nulls   :  6974
num_of_uniques :  434
--------------------------------
NaN       6974
1163.0     574
1360.0     356
1165.0     301
1335.0     242
          ... 
1960.0       1
1258.0       1
1167.0       1
1331.0       1
2037.0       1
Name: Weight_kg, Length: 435, dtype: int64


In [157]:
df["Weight_kg"].fillna("-", inplace=True)

In [158]:
df.groupby(["make_model", "body_type","Weight_kg"]).price.describe()

count          mean          std      min  \
make_model     body_type Weight_kg                                              
Audi A1        Compact   102.0        1.0  19229.000000          NaN  19229.0   
                         1010.0       2.0  15450.000000   707.106781  14950.0   
                         1035.0       6.0  16796.666667  2617.866816  14390.0   
                         1040.0       2.0  20424.500000  2933.786035  18350.0   
                         1065.0      36.0  20971.777778  1982.553564  15500.0   
...                                   ...           ...          ...      ...   
Renault Espace Van       2037.0       1.0  47950.000000          NaN  47950.0   
                         2353.0       1.0  22990.000000          NaN  22990.0   
                         2410.0       1.0  23990.000000          NaN  23990.0   
                         2471.0       5.0  24738.000000  8470.638701  17400.0   
                         -          377.0  30126.297082  7273.476242  12614.0   

                                         25%      50%       75%      max  
make_model     body_type Weight_kg                                        
Audi A1        Compact   102.0      19229.00  19229.0  19229.00  19229.0  
                         1010.0     15200.00  15450.0  15700.00  15950.0  
                         1035.0     15892.50  15900.0  16575.00  21900.0  
                         1040.0     19387.25  20424.5  21461.75  22499.0  
                         1065.0     18987.50  21690.0  22400.00  23550.0  
...                                      ...      ...       ...      ...  
Renault Espace Van       2037.0     47950.00  47950.0  47950.00  47950.0  
                         2353.0     22990.00  22990.0  22990.00  22990.0  
                         2410.0     23990.00  23990.0  23990.00  23990.0  
                         2471.0     20900.00  20900.0  25500.00  38990.0  
                         -          24990.00  29990.0  34950.00  68320.0  

[894 rows x 8 columns]

In [159]:
df["Weight_kg"].replace("-", np.nan, inplace=True)

In [160]:
fill(df, "make_model", "body_type", "Weight_kg", "mode")

Number of NaN :  0
------------------
1163.0    1582
1360.0    1419
1487.0     966
1135.0     837
1425.0     744
          ... 
1331.0       1
1132.0       1
1252.0       1
1792.0       1
2037.0       1
Name: Weight_kg, Length: 434, dtype: int64


In [161]:
first_looking("CO2_Emission")

column name    :  CO2_Emission
--------------------------------
per_of_nulls   :  % 15.3
num_of_nulls   :  2436
num_of_uniques :  120
--------------------------------
NaN        2436
120.000     740
99.000      545
97.000      537
104.000     501
           ... 
990.000       1
12.324        1
1.060         1
51.000        1
193.000       1
Name: CO2_Emission, Length: 121, dtype: int64


In [162]:
df["CO2_Emission"].fillna("-", inplace=True)

In [163]:
df.groupby(["make_model", "body_type","CO2_Emission"]).price.describe()

count          mean          std  \
make_model     body_type CO2_Emission                                     
Audi A1        Compact   1.0             1.0  20881.000000          NaN   
                         9.0             1.0  12479.000000          NaN   
                         10.0          147.0  16946.870748  2850.063119   
                         11.0           25.0  19960.360000  2676.261601   
                         14.0            2.0  28750.000000   268.700577   
...                                      ...           ...          ...   
Renault Espace Van       188.0           1.0  43080.000000          NaN   
                         191.0           2.0  28465.000000     0.000000   
                         193.0           1.0  38990.000000          NaN   
                         194.0           1.0  46490.000000          NaN   
                         -             209.0  31038.459330  8711.240792   

                                           min      25%      50%      75%  \
make_model     body_type CO2_Emission                                       
Audi A1        Compact   1.0           20881.0  20881.0  20881.0  20881.0   
                         9.0           12479.0  12479.0  12479.0  12479.0   
                         10.0           9950.0  15765.0  16450.0  18333.0   
                         11.0          14930.0  17970.0  19990.0  21460.0   
                         14.0          28560.0  28655.0  28750.0  28845.0   
...                                        ...      ...      ...      ...   
Renault Espace Van       188.0         43080.0  43080.0  43080.0  43080.0   
                         191.0         28465.0  28465.0  28465.0  28465.0   
                         193.0         38990.0  38990.0  38990.0  38990.0   
                         194.0         46490.0  46490.0  46490.0  46490.0   
                         -             12990.0  24750.0  29990.0  34200.0   

                                           max  
make_model     body_type CO2_Emission           
Audi A1        Compact   1.0           20881.0  
                         9.0           12479.0  
                         10.0          29150.0  
                         11.0          28980.0  
                         14.0          28940.0  
...                                        ...  
Renault Espace Van       188.0         43080.0  
                         191.0         28465.0  
                         193.0         38990.0  
                         194.0         46490.0  
                         -             64332.0  

[760 rows x 8 columns]

In [164]:
df["CO2_Emission"].replace("-", np.nan, inplace=True)

In [165]:
fill(df, "make_model", "body_type", "CO2_Emission", "median")

Number of NaN :  0
------------------
120.00    836
104.00    679
106.00    674
114.00    556
99.00     546
         ... 
1.06        1
51.00       1
165.00      1
112.50      1
193.00      1
Name: CO2_Emission, Length: 123, dtype: int64


In [166]:
first_looking("Comfort_Convenience")

column name    :  Comfort_Convenience
--------------------------------
per_of_nulls   :  % 5.78
num_of_nulls   :  920
num_of_uniques :  6198
--------------------------------
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                               

In [167]:
first_looking("Entertainment_Media")

column name    :  Entertainment_Media
--------------------------------
per_of_nulls   :  % 8.63
num_of_nulls   :  1374
num_of_uniques :  346
--------------------------------
NaN                                                                                                 1374
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                          1282
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                       982
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                             783
On-board computer,Radio                                                                              487
                                                                                                    ... 
Bluetooth,CD player,Digital radio,Hands-free equipment,Radio                                           1
CD player,Hands-free equipment,MP3,Radio,USB                                               

In [168]:
fill(df, "make_model", "body_type", "Entertainment_Media", "mode")

Number of NaN :  0
------------------
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                      1738
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                  1134
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB        1010
On-board computer                                                                615
Radio                                                                            558
                                                                                ... 
Bluetooth,CD player,MP3                                                            1
CD player,USB                                                                      1
Bluetooth,CD player,Digital radio,Radio,USB                                        1
Bluetooth,CD player,Digital radio,MP3,On-board computer,Radio,Television,USB       1
Hands-free equipment,On-board computer,Radio,Sound system                          1
Name: Entertainment_Media, 

In [169]:
first_looking("Extras")

column name    :  Extras
--------------------------------
per_of_nulls   :  % 18.61
num_of_nulls   :  2962
num_of_uniques :  659
--------------------------------
Alloy wheels                                                                                               3245
NaN                                                                                                        2962
Alloy wheels,Touch screen                                                                                   697
Alloy wheels,Voice Control                                                                                  577
Alloy wheels,Touch screen,Voice Control                                                                     541
                                                                                                           ... 
Alloy wheels,Catalytic Converter,Shift paddles,Sport package,Sport seats,Sport suspension,Voice Control       1
Alloy wheels,Catalytic Converter,Roof rack,Sport packa

In [170]:
fill(df, "make_model", "body_type", "Extras", "mode")

Number of NaN :  0
------------------
Alloy wheels                                                                                               5786
Alloy wheels,Touch screen                                                                                   697
Roof rack                                                                                                   596
Alloy wheels,Voice Control                                                                                  582
Alloy wheels,Touch screen,Voice Control                                                                     544
                                                                                                           ... 
Alloy wheels,Catalytic Converter,Shift paddles,Sport package,Sport seats,Sport suspension,Voice Control       1
Alloy wheels,Catalytic Converter,Roof rack,Sport package,Sport seats,Trailer hitch                            1
Alloy wheels,Catalytic Converter,Ski bag,Sport package,Voice Contr

In [171]:
first_looking("Safety_Security")

column name    :  Safety_Security
--------------------------------
per_of_nulls   :  % 6.17
num_of_nulls   :  982
num_of_uniques :  4443
--------------------------------
NaN                                                                                                                                                                                                                                                                                                                                                               982
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                      538
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Pa

In [172]:
fill(df, "make_model", "body_type", "Safety_Security", "mode")

Number of NaN :  0
------------------
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                      729
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                                                 480
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction contro

In [173]:
first_looking("cons_comb")

column name    :  cons_comb
--------------------------------
per_of_nulls   :  % 12.77
num_of_nulls   :  2033
num_of_uniques :  72
--------------------------------
NaN     2033
5.4      770
3.9      733
4.0      713
5.1      657
        ... 
33.0       1
50.0       1
1.0        1
46.0       1
11.0       1
Name: cons_comb, Length: 73, dtype: int64


In [174]:
cons_comb = (df["cons_country"] + df["cons_city"])/2

In [175]:
df["cons_comb"] = df["cons_comb"].fillna(cons_comb)

In [176]:
df["cons_comb"].value_counts(dropna=False)

NaN     1925
5.4      770
3.9      733
4.0      713
5.1      657
        ... 
33.0       1
50.0       1
1.0        1
46.0       1
13.8       1
Name: cons_comb, Length: 79, dtype: int64

In [177]:
df["cons_comb"].fillna("-", inplace=True)

In [178]:
df.groupby(["make_model", "body_type","cons_comb"]).price.describe()

count          mean          std      min  \
make_model     body_type cons_comb                                              
Audi A1        Compact   3.0          4.0  14370.250000  4481.392669  11100.0   
                         3.4          6.0  12755.000000   900.749688  11800.0   
                         3.5          3.0  14613.000000  1848.098212  12479.0   
                         3.6          5.0  13960.000000  1232.071427  12900.0   
                         3.7         50.0  15902.200000  2960.324058  12900.0   
...                                   ...           ...          ...      ...   
Renault Espace Van       6.8        115.0  34032.313043  7847.720221  25490.0   
                         7.4         61.0  38163.754098  6406.548058  28975.0   
                         7.6         14.0  52019.214286  7596.268478  38990.0   
                         46.0         1.0  25490.000000          NaN  25490.0   
                         -          144.0  29865.520833  6616.406401  12990.0   

                                        25%      50%       75%      max  
make_model     body_type cons_comb                                       
Audi A1        Compact   3.0        11242.5  12840.5  15968.25  20700.0  
                         3.4        11987.5  12640.0  13570.00  13800.0  
                         3.5        14079.5  15680.0  15680.00  15680.0  
                         3.6        12900.0  13900.0  14200.00  15900.0  
                         3.7        13900.0  15040.0  15967.50  23500.0  
...                                     ...      ...       ...      ...  
Renault Espace Van       6.8        28237.5  31990.0  37470.00  58950.0  
                         7.4        33781.0  37475.0  39990.00  54500.0  
                         7.6        48387.5  51350.0  54982.00  64332.0  
                         46.0       25490.0  25490.0  25490.00  25490.0  
                         -          24497.5  29990.0  33525.00  63477.0  

[582 rows x 8 columns]

In [179]:
df["cons_comb"].replace([0.0, 1.0, 1.2, 1.6, 10, 11, 13.8, 32.0, 33.0, 38.0, 40.0, 43.0, 46.0, 50.0, 51.0, 54.0, 55.0, "-"], np.nan, inplace=True)

In [180]:
df["cons_comb"].value_counts(dropna=False)

NaN     1983
5.40     770
3.90     733
4.00     713
5.10     657
        ... 
5.45       2
8.10       2
7.50       1
9.10       1
4.55       1
Name: cons_comb, Length: 62, dtype: int64

In [181]:
fill(df, "make_model", "body_type", "cons_comb", "median")

Number of NaN :  0
------------------
4.00    1020
5.60     965
5.40     900
5.00     810
4.70     766
        ... 
8.10       2
4.55       2
7.50       1
9.10       1
4.30       1
Name: cons_comb, Length: 63, dtype: int64


In [182]:
df.drop("cons_country", axis = 1, inplace = True)

In [183]:
df.drop("cons_city", axis = 1, inplace = True)

In [184]:
df.shape

(15919, 26)

In [185]:
df.isnull().sum()/df.shape[0]*100

make_model             0.000000
body_type              0.000000
price                  0.000000
vat                    0.000000
km                     0.000000
Type                   0.000000
Fuel                   0.000000
Gears                  0.000000
Comfort_Convenience    5.779257
Entertainment_Media    0.000000
Extras                 0.000000
Safety_Security        0.000000
age                    0.000000
Previous_Owners        0.000000
hp_kW                  0.000000
Inspection_new         0.000000
Paint_Type             0.000000
Upholstery_type        0.000000
Nr_of_Doors            0.000000
Nr_of_Seats            0.000000
Gearing_Type           0.000000
Displacement_cc        0.000000
Weight_kg              0.000000
Drive_chain            0.000000
cons_comb              0.000000
CO2_Emission           0.000000
dtype: float64

In [186]:
df.to_csv("filled_scout.csv", index=False)